In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os 
import subprocess
import sys
import gc

# pickle
import pickle

from src.hVOS.cell import Cell

In [ ]:
run_id = 1

data_dir = 'C:/Users/jjudge3/Desktop/neuron docker/'
compart_data = {}
should_create_mem_map = False  # if True, create mem mapped files. If False, load mem mapped files

In [3]:
# extract all the .tar.gz files

target_dir = data_dir + 'archive/run' + str(run_id) + "/"
for file in os.listdir(data_dir):
    compart = file.replace('.tar.gz', "").replace('S1-Thal-output-',"")
    output_dir = target_dir + compart + '/'
    
    if file.endswith('.tar.gz'):
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            result = subprocess.run(['tar', '-xzvf', data_dir + file, "-C", output_dir], capture_output=True, text=True, check=True)
            
            print('Extracted ' + file)
        # else it already exists and was extracted
        #os.rename(data_dir + file, data_dir + 'archive/' + file)
        compart_data[compart] = output_dir
compart_data

{'apic_0_10': 'C:/Users/jjudge3/Desktop/neuron docker/archive/run1/apic_0_10/',
 'apic_100_140': 'C:/Users/jjudge3/Desktop/neuron docker/archive/run1/apic_100_140/',
 'apic_10_20': 'C:/Users/jjudge3/Desktop/neuron docker/archive/run1/apic_10_20/',
 'apic_140_180': 'C:/Users/jjudge3/Desktop/neuron docker/archive/run1/apic_140_180/',
 'apic_20_30': 'C:/Users/jjudge3/Desktop/neuron docker/archive/run1/apic_20_30/',
 'apic_30_40': 'C:/Users/jjudge3/Desktop/neuron docker/archive/run1/apic_30_40/',
 'apic_40_60': 'C:/Users/jjudge3/Desktop/neuron docker/archive/run1/apic_40_60/',
 'apic_60_80': 'C:/Users/jjudge3/Desktop/neuron docker/archive/run1/apic_60_80/',
 'apic_80_100': 'C:/Users/jjudge3/Desktop/neuron docker/archive/run1/apic_80_100/',
 'axon_0_10': 'C:/Users/jjudge3/Desktop/neuron docker/archive/run1/axon_0_10/',
 'dend_0_10': 'C:/Users/jjudge3/Desktop/neuron docker/archive/run1/dend_0_10/',
 'dend_100_140': 'C:/Users/jjudge3/Desktop/neuron docker/archive/run1/dend_100_140/',
 'dend_1

In [16]:
# load one pkl to look at cell data
cell_id_to_me_type_map = {}

# 
me_type_map_file = 'C:/Users/jjudge3/Desktop/neuron docker/' + 'archive/run' + str(run_id) + "/" + 'cell_id_to_me_type_map.pkl'

if os.path.exists(me_type_map_file):
    with open(me_type_map_file, 'rb') as f:
        cell_id_to_me_type_map = pickle.load(f)
else:

    compart = list(compart_data.keys())[0]
    target_dir_net = compart_data[compart] + '/S1_Thal_NetPyNE_Frontiers_2022/data/v7_batch1/v7_batch1_0_0_data.pkl'

    with open(target_dir_net, 'rb') as f:
        data = pickle.load(f)
        print(data.keys())
        for cell_dict in data['net']['cells']:
            cell_id_to_me_type_map[cell_dict['gid']] = {
                'me_type': cell_dict['tags']['cellType'],
                'x': cell_dict['tags']['x'],
                'y': cell_dict['tags']['y'],
                'z': cell_dict['tags']['z']
            }

    with open(me_type_map_file, 'wb') as f:
        pickle.dump(cell_id_to_me_type_map, f)

dict_keys(['netpyne_version', 'netpyne_changeset', 'net', 'simConfig', 'simData'])


In [ ]:
loaded_compart_data = {}
time = None
if should_create_mem_map:
    for compart in compart_data.keys():
        print(compart)
        if compart in loaded_compart_data:
            continue
        target_dir = compart_data[compart]
        if not os.path.exists(target_dir):
            print('Directory ' + target_dir + ' does not exist')
            continue
        target_dir += '/S1_Thal_NetPyNE_Frontiers_2022/data/v7_batch1/'
        data_file = target_dir + 'v7_batch1_0_0_data.pkl'

        if not os.path.exists(data_file):
            print("Data file not found:", data_file)
            continue
        # also skip if the .dat files already exist
        mmaps_exist = False
        for file in os.listdir(target_dir):
            if file.endswith('.dat') and 'v7_batch1_0_0' in file and 'cell' in file:
                print("\t", target_dir + file)
                mmaps_exist = True
                break
        if mmaps_exist:
            print("Memory mapped files already exist for compartment", compart)
            continue

        # Load data
        with open(data_file, 'rb') as f:
            try:
                data = pickle.load(f)
            except MemoryError:
                print("MemoryError with file:", data_file)
                raise MemoryError
            print(data.keys())
            if time is None:  # store t only once
                time = np.array(data['simData']['t'])

            for k in data['simData']:
                if compart[:4] in k:
                    for cell_id in data['simData'][k]:
                        print(k, cell_id)

                        # create memory mapped file for this cell's segment data
                        mm_data_fp = target_dir + 'v7_batch1_0_0_' + k + '_' + cell_id + '.dat'
                        cell_data = np.array(data['simData'][k][cell_id])

                        # create empty file if it doesn't exist
                        if not os.path.exists(mm_data_fp):
                            with open(mm_data_fp, 'wb') as f:
                                f.write(b'\0' * cell_data.nbytes)

                        # store as a memory mapped array for faster access / less memory usage
                        mm_data_fp = np.memmap(mm_data_fp, dtype='float32', mode='w+', shape=cell_data.shape)
                        mm_data_fp[:] = cell_data[:]
                        mm_data_fp.flush()
                        if compart not in loaded_compart_data:
                            loaded_compart_data[compart] = {}
                        if cell_id not in loaded_compart_data[compart]:
                            loaded_compart_data[compart][cell_id] = {}
                        loaded_compart_data[compart][cell_id] = mm_data_fp  # store just pointer
            # to avoid memory issues, delete data after it's been stored
            del data
            gc.collect()

    # also store time to memory mapped file
    if time is not None:
        target_dir = data_dir + 'archive/run' + str(run_id) + '/'
        mm_time_fp = target_dir + 'v7_batch1_0_0_time.dat'
        if not os.path.exists(mm_time_fp):
            with open(mm_time_fp, 'wb') as f:
                f.write(b'\0' * time.nbytes)
        mm_time_fp = np.memmap(mm_time_fp, dtype='float32', mode='w+', shape=time.shape)
        mm_time_fp[:] = time[:]
        mm_time_fp.flush()
        loaded_compart_data['time'] = mm_time_fp  # store just pointer
else: # load mem mapped files. Get file pointers from looking through data_dir dat files and load them into loaded_compart_data
    for compart in compart_data.keys(): 
        target_dir = data_dir + 'archive/run' + str(run_id) + '/' + compart
        if not os.path.exists(target_dir):
            print('Directory ' + target_dir + ' does not exist')
            continue
        target_dir += '/S1_Thal_NetPyNE_Frontiers_2022/data/v7_batch1/'
        for file in os.listdir(target_dir):
            if file.endswith('.dat') and 'v7_batch1_0_0' in file and 'cell' in file:
                file_name = file.replace(".dat", "").replace("v7_batch1_0_0_", "").split('_')
                compart = "_".join(file_name[:2])
                cell_id = "_".join(file_name[2:])
                if 'soma' in file:
                    compart = 'Vsoma'
                    cell_id = 'cell_' + cell_id
                    
                if compart not in loaded_compart_data:
                    loaded_compart_data[compart] = {}
                if cell_id not in loaded_compart_data[compart]:
                    loaded_compart_data[compart][cell_id] = {}
                loaded_compart_data[compart][cell_id] = target_dir + file
    # load time
    target_dir = data_dir + 'archive/run' + str(run_id) + '/'
    mm_time_fp = target_dir + 'v7_batch1_0_0_time.dat'
    loaded_compart_data['time'] = np.memmap(mm_time_fp, dtype='float32', mode='r')

loaded_compart_data

In [ ]:
# for each soma, get a cell id and aggregate its axons, apics, dends
cells = {}
for cell_id in loaded_compart_data['Vsoma']:
    soma = loaded_compart_data['Vsoma'][cell_id]
    axons = {}
    apics = {}
    dends = {}
    for compart in loaded_compart_data.keys():
        if compart == 'time':
            continue
        if cell_id in loaded_compart_data[compart]:
            if 'axon' in compart:
                axons[compart] = loaded_compart_data[compart][cell_id]
            elif 'apic' in compart:
                apics[compart] = loaded_compart_data[compart][cell_id]
            elif 'dend' in compart:
                dends[compart] = loaded_compart_data[compart][cell_id]
    cells[cell_id] = Cell(cell_id, None, None, axons, apics, dends, soma)

cells

In [ ]:
# choose a random cell to plot
rand_cell_id = 'cell_21986' #np.random.choice(list(cells.keys()))
rand_cell = cells[rand_cell_id]

apics = rand_cell.get_apic_filemap()
dends = rand_cell.get_dend_filemap()
axons = rand_cell.get_axon_filemap()
time = loaded_compart_data['time']

# plot trace for cell 
plt.figure()

handles = [None, None, None, None]
for apic_comp in apics:
    handles[1] = plt.plot(time, rand_cell.get_voltage_trace(compart_id=apic_comp),  c='r', alpha=0.5)
for dend_comp in dends:
    handles[2] = plt.plot(time, rand_cell.get_voltage_trace(compart_id=dend_comp),  c='g', alpha=0.5)
for axon_comp in axons:
    handles[3] = plt.plot(time, rand_cell.get_voltage_trace(compart_id=axon_comp),  c='b', alpha=0.5)
handles[0] = plt.plot(time, rand_cell.get_soma_voltage_trace(), c='k', linewidth=2)

labels = ['Soma', 'Apic', 'Dend', 'Axon']
labels = [labels[i] for i in range(len(handles)) if handles[i] is not None]
handles = [h[0] for h in handles if h is not None]

plt.legend(handles=handles, labels=['Soma', 'Apic', 'Dend', 'Axon'])
plt.xlabel('Time (ms)')
plt.ylabel('Voltage (mV)')
plt.title('Compartment Voltages for ' + rand_cell_id)
plt.savefig('comp_voltage_' + rand_cell_id + '.png')
print(os.getcwd() + 'comp_voltage_' + rand_cell_id + '.png')
plt.show()

In [ ]:
spkt = data['simData']['spkt']
spkid = data['simData']['spkid']
Vapic_0 = data['simData']['Vapic_0']
print('spkt:', spkt)
print('spkid:', spkid)
print('Vapic_0:', Vapic_0)

In [ ]:
n_traces = 0
for k in data['simData'].keys():
    if k.startswith("V") and "_" in k:
        #print(k)
        #print("\t", len(data['simData'][k]))
        n_traces += len(data['simData'][k])
print("n_traces:", n_traces)